In [4]:
import numpy as np
import pandas as pd
import random

In [5]:
from sklearn.preprocessing import OneHotEncoder


def get_train_data():
    data_train = pd.read_csv('../Datasets/Fashion MNIST/fashion-mnist_train.csv')
    X_train = data_train.drop(['label'], axis=1).to_numpy()
    X_train = X_train.reshape((X_train.shape[0], 28, 28))

    y_train = data_train['label'].to_numpy().reshape(-1, 1)  # Изменение формы массива
    encoder = OneHotEncoder(sparse=False)
    y_train_one_hot = encoder.fit_transform(y_train)

    return X_train / 255, y_train_one_hot

In [6]:
X_train, y_train = get_train_data()

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
X_train

In [ ]:
y_train

In [13]:
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import OneHotEncoder


def get_train_data():
    data_train = pd.read_csv('../Datasets/Fashion MNIST/fashion-mnist_train.csv')
    X_train = data_train.drop(['label'], axis=1).to_numpy()
    X_train = X_train.reshape((X_train.shape[0], 28, 28))

    y_train = data_train['label'].to_numpy().reshape(-1, 1)  # Изменение формы массива
    encoder = OneHotEncoder(sparse=False)
    y_train_one_hot = encoder.fit_transform(y_train)

    return X_train / 255, y_train_one_hot


class CNN:
    def __init__(self, input_size, num_classes):
        self.input_size = input_size
        self.num_classes = num_classes
        self.filters1 = np.random.randn(3, 3, 1, 32)
        self.filters2 = np.random.randn(3, 3, 32, 64)
        self.weights_fc = np.random.randn(7 * 7 * 64, 100)
        self.weights_output = np.random.randn(100, num_classes)

    def convolve2d(self, input_data, filters):
        _, height, width, depth = input_data.shape
        filter_height, filter_width, _, num_filters = filters.shape
    
        output_height = height - filter_height + 1
        output_width = width - filter_width + 1
    
        output_data = np.zeros((output_height, output_width, num_filters))
    
        for i in range(output_height):
            for j in range(output_width):
                output_data[i, j, :] = np.sum(
                    input_data[i:i + filter_height, j:j + filter_width, :, np.newaxis] * filters,
                    axis=(0, 1, 2, 3)
                )
    
        return output_data

    def max_pooling(self, input_data, pool_size=(2, 2)):
        print('input_data.shape', input_data.shape)
        print('pool_size', pool_size)
        height, width, num_filters = input_data.shape
        pool_height, pool_width = pool_size
        
    
        output_height = height // pool_height
        output_width = width // pool_width
    
        output_data = np.zeros((output_height, output_width, num_filters))
    
        for i in range(output_height):
            for j in range(output_width):
                output_data[i, j, :] = np.max(
                    input_data[i * pool_height:(i + 1) * pool_height, j * pool_width:(j + 1) * pool_width],
                    axis=(0, 1)
                )
    
        return output_data

    def flatten(self, input_data):
        return input_data.reshape(-1)

    def fully_connected(self, input_data, weights):
        return np.dot(input_data, weights)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x))
        return exp_x / exp_x.sum(axis=0, keepdims=True)

    def train(self, X_train, y_train, learning_rate=0.01, epochs=10):
        for epoch in range(epochs):
            for index, image in enumerate(X_train):
                # Расширение размерности для корректного проведения операции свертки
                image = image.reshape((1, 28, 28, 1))
                conv1_output = self.convolve2d(image, self.filters1)
                pool1_output = self.max_pooling(conv1_output)

                conv2_output = self.convolve2d(pool1_output, self.filters2)
                pool2_output = self.max_pooling(conv2_output)

                flattened_output = self.flatten(pool2_output)
                fc_output = self.fully_connected(flattened_output, self.weights_fc)
                fc_output_relu = np.maximum(0, fc_output)

                output = self.fully_connected(fc_output_relu, self.weights_output)
                predicted_probs = self.softmax(output)

                loss_gradient = predicted_probs - y_train[index]
                weights_output_gradient = np.outer(fc_output_relu, loss_gradient)
                fc_output_gradient = np.dot(self.weights_output, loss_gradient)
                fc_output_gradient[fc_output < 0] = 0
                weights_fc_gradient = np.outer(flattened_output, fc_output_gradient)

                self.weights_output -= learning_rate * weights_output_gradient
                self.weights_fc -= learning_rate * weights_fc_gradient.reshape(self.weights_fc.shape)

    def predict(self, X_test):
        predictions = []
        for image in X_test:
            # Расширение размерности для корректного проведения операции свертки
            image = image.reshape((1, 28, 28, 1))
            conv1_output = self.convolve2d(image, self.filters1)
            pool1_output = self.max_pooling(conv1_output)

            conv2_output = self.convolve2d(pool1_output[np.newaxis, :], self.filters2)
            pool2_output = self.max_pooling(conv2_output)

            flattened_output = self.flatten(pool2_output)
            fc_output = self.fully_connected(flattened_output, self.weights_fc)
            fc_output_relu = np.maximum(0, fc_output)

            output = self.fully_connected(fc_output_relu, self.weights_output)
            predictions.append(output)

        return predictions


if __name__ == '__main__':
    X_train, y_train = get_train_data()
    input_shape = X_train.shape
    num_classes = 10
    simple_conv_net = CNN(input_shape, num_classes)
    simple_conv_net.train(X_train, y_train)
    # predictions = simple_conv_net.predict(X_test)


D:\PycharmProjects\Design_and_training_neural_networks\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ValueError: operands could not be broadcast together with shapes (1,3,28,1,1) (3,3,1,32) 